**This notebook is a practice lab in the [Intro to Game AI and Reinforcement Learning](https://www.kaggle.com/learn/intro-to-game-ai-and-reinforcement-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/deep-reinforcement-learning).**

---


# Introduction

In this practice lab I want to train an agent against different opponent agents. My aims are
- to understand how the performance changes after each training against a specific agent
- to understand how the overall performance changes across all trainings against all agents
- to benchmark the agent against a very strong opponent. See the [collection](connectX_agents_collection) gathered from [vyacheslavbolotin](https://www.kaggle.com/vyacheslavbolotin)'s [overview](https://www.kaggle.com/code/vyacheslavbolotin/agents-connect-x)
- to potentially test various models
- to potentially test various reward schemes (I liked the idea of [Pascal Pons](http://blog.gamesolver.org/solving-connect-four/01-introduction/) to take the number of moves required to win the game into account)
- to apply the framework of my [practice lab cart pole](practice_lab_stable_baselines3_cartpole_v1.ipynb)

## Approach

1. I want to develop the solution step-wise starting with the "prooven" approach from the Kaggle course.
2. I'll add TensorBoard to better monitor the training. 

# Basic considerations

### 1. Set the architecture (tbd)




Here I'll add some thoughts about the model, its architecture, and the policy.

### 2. Decide reward

#### Option 1: [Pascal Pons](http://blog.gamesolver.org/solving-connect-four/02-test-protocol/)
We define a score for any non final position reflecting the outcome of the game for the player to play, considering that both players play perfectly and try to win as soon as possible or lose as late as possible. A position has:
- a positive score if the current player can win. 1 if it wins with the last stone, 2 if it wins with your second last stone and so on…
- a null score if the game will end by a draw game
- a negative score if the current player lose whatever it plays. -1 if the opponent wins with the last stone, -2 if the opponent wins with the second last stone and so on…

#### Option 2: [Kaggle](https://www.kaggle.com/code/alexisbcook/deep-reinforcement-learning)
- If the agent wins the game in that move, we give it a reward of +1.
- Else if the agent plays an invalid move (which ends the game), we give it a reward of -10.
- Else if the opponent wins the game in its next move (i.e., the agent failed to prevent its opponent from winning), we give the agent a reward of -1.
- Else, the agent gets a reward of 1/42.

# Basic environment, agents playing the game and evaluation

In [10]:
# imports
from kaggle_environments import make, evaluate
import numpy as np

In [6]:
# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run(["random", "random"])

# Enjoy the game
env.render(mode="ipython")

In [29]:
# Evaluation function including print
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Draw Percentage:", np.round(outcomes.count([0,0])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
# Parameters for evaluation
agent1="random"
agent2="negamax"
n_rounds=100

# Call evaluation function
get_win_percentages(agent1, agent2, n_rounds)

Agent 1 Win Percentage: 0.01
Agent 2 Win Percentage: 0.99
Draw Percentage: 0.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


[[-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [1, -1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, 1]]

# Custom environment and neural network

In [4]:
# original code from the exercise

import random
import numpy as np
import pandas as pd
import gym
import matplotlib.pyplot as plt
%matplotlib inline

from kaggle_environments import make, evaluate
from gym import spaces

class ConnectFourGym(gym.Env):
    def __init__(self, agent2="random"):
        ks_env = make("connectx", debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(1,self.rows,self.columns), dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    def reset(self):
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns)
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns), reward, done, _
    
# Create ConnectFour environment 
env = ConnectFourGym(agent2="random")

import torch as th
import torch.nn as nn

!pip install "stable-baselines3"
from stable_baselines3 import PPO 
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Neural network for predicting action values
class CustomCNN(BaseFeaturesExtractor):
    
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int=128):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # CxHxW images (channels first)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
)

ModuleNotFoundError: No module named 'gym'

In [45]:
# TODO make switch to gymnasium (read this https://gymnasium.farama.org/introduction/migration_guide/)
# TODO run a short training against the "random"
# TODO test multicore evaluation

import random
import numpy as np
import pandas as pd
import gymnasium as gym
import matplotlib.pyplot as plt
%matplotlib inline

from kaggle_environments import make, evaluate
from gymnasium import spaces

class ConnectFourGym(gym.Env):
    def __init__(self, agent2="random"):
        ks_env = make("connectx", debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(1,self.rows,self.columns), dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
    def reset(self):
        # try for gym to gymnasium transition
        super().reset(seed=42)
        self.obs = self.env.reset()
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns)
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            # line Kaggle o-code: self.obs, old_reward, done, _ = self.env.step(int(action))
            # changed line to migrate from gym to gymnasium
            self.obs, old_reward, terminated, truncated, _ = self.env.step(int(action))
            # additional line to migrate from gym to gymnasium
            done = terminated or truncated
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
        return np.array(self.obs['board']).reshape(1,self.rows,self.columns), reward, done, _
    
# Create ConnectFour environment 
env = ConnectFourGym(agent2="random")

import torch as th
import torch.nn as nn

# !pip install "stable-baselines3"
from stable_baselines3 import PPO 
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Neural network for predicting action values
class CustomCNN(BaseFeaturesExtractor):
    
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int=128):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # CxHxW images (channels first)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
)

Next, run the code cell below to train an agent with PPO.  This code is identical to the code from the tutorial.

In [46]:
# Initialize agent
model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

# Train agent
model.learn(total_timesteps=1000)

TypeError: ConnectFourGym.reset() got an unexpected keyword argument 'seed'

# Opponents for training and evaluation

## Agent with n-step lookahead and alpha-beta pruning

In [2]:
# agent with n-step lookahead and alpha-beta pruning
def my_nstep_lookahead_ab_pruning_agent(obs, config):
    # Your code here: Amend the agent!

    import random
    import numpy as np
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
        
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows

    # Helper function for get_heuristic: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)
        score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score

    # Uses minimax with alpha-beta pruning to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps, alpha=-float('inf'), beta=float('inf')):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, False, mark, config, alpha, beta)
        return score
    
    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow
    
    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False
    
    # Minimax implementation with alpha-beta pruning
    def minimax(node, depth, maximizingPlayer, mark, config, alpha, beta):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
        if maximizingPlayer:
            value = -np.inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth-1, False, mark, config, alpha, beta))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, minimax(child, depth-1, True, mark, config, alpha, beta))
                beta = min(beta, value)
                if alpha >= beta:
                    break
            return value

    # agent driver
    # How deep to make the game tree: higher values take longer to run!
    # ConncectX comes with a max time per player move and also for all moves!
    N_STEPS = 5
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

# Training

In [2]:
# Initialize agent
model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

# Train agent
model.learn(total_timesteps=50000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


# Trained agent

In [3]:
def agent1(obs, config):
    # Use the best model to select a column
    col, _ = model.predict(np.array(obs['board']).reshape(1, 6,7))
    # Check if selected column is valid
    is_valid = (obs['board'][int(col)] == 0)
    # If not valid, select random move. 
    if is_valid:
        return int(col)
    else:
        return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])

# Evaluation

In [4]:
# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agent1, my_nstep_lookahead_ab_pruning_agent])

# Show the game
env.render(mode="ipython")

NameError: name 'make' is not defined

In [8]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [11]:
get_win_percentages(agent1=agent1, agent2="negamax")

Agent 1 Win Percentage: 0.06
Agent 2 Win Percentage: 0.93
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


In [ ]:
import numpy as np
from multiprocessing import Pool

def get_win_percentages_mc(agent1, agent2, n_rounds=100, num_cores=6):
    """
    Calculates win percentages for two agents in a Connect Four game.

    Args:
        agent1: The first agent.
        agent2: The second agent.
        n_rounds: The total number of games to play.
        num_cores: The number of CPU cores to use for parallel processing.

    Returns:
        None
    """

    config = {'rows': 6, 'columns': 7, 'inarow': 4}

    # Calculate the number of games per core
    games_per_core = n_rounds // (2 * num_cores) 
    remaining_games = n_rounds % (2 * num_cores)

    # Create a list of arguments for each process
    args_list = []
    for i in range(num_cores):
        # Alternate agent order for each core
        rounds = games_per_core
        if i < remaining_games:
            rounds += 1
        args_list.append(("connectx", [agent1, agent2], config, [], rounds))
        args_list.append(("connectx", [agent2, agent1], config, [], rounds))

    # Create a pool of worker processes
    with Pool(processes=num_cores) as pool:
        # Run the evaluation function in parallel
        results = pool.starmap(evaluate, args_list)

    # Combine the results from all cores
    outcomes = []
    for core_results in results:
        outcomes.extend(core_results)

    # Calculate and print win percentages and invalid play counts
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1, -1]) / len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1, 1]) / len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
import numpy as np
get_win_percentages_mc(agent1=agent1, agent2=my_4step_lookahead_ab_pruning_agent)

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-game-ai-and-reinforcement-learning/discussion) to chat with other learners.*